In [1]:
from experiments.experiments_results import get_experiments_data, generate_df_from_experiments
from utils.utils import get_list_of_users
import pandas as pd

In [2]:
generate_df_from_experiments()

This took 0.123


In [16]:
def get_results_from_keywords(comp_col='arch', based_on='score', ix=-1, **kwargs):
    '''
    This function generates a table that ranks the specify comp_col columns
    based on its performance (mean_score col) for all the users
    
    '''
    df = get_experiments_data()
    
    col_values = list(df[comp_col].drop_duplicates())
    nb_values = len(col_values)
    rank_col_names = [f'rank{i}' for i in range(1,nb_values+1)]
    
    if ix>=0:
        based_on = f'{based_on}_{ix}'
    else: based_on = f'mean_{based_on}'
        
    
    assert all(k in df.columns for k in kwargs.keys()) , f'kwargs must be one of {df.columns}'
    for k,v in kwargs.items():
        df = df.loc[df[k]==v]
    print(kwargs)
    print(f'Remaining experiments: {df.shape[0]}')
    
    rows = []
    for user in get_list_of_users():
        sorted_scores = df.loc[(df.user==user)][[comp_col,based_on]].sort_values(based_on, ascending=True).drop_duplicates(subset=[comp_col])
        best =  sorted_scores.iloc[0:nb_values,0].values
        row = {'user': user, 'best': best}
        rows.append(row)

    results = pd.DataFrame(rows)
    results[rank_col_names] = pd.DataFrame(results.best.tolist(), index=results.index)
    del results['best']
    
    summarize = pd.DataFrame(columns=results.columns, index=col_values)
    for i in summarize.columns:
        for j in summarize.index.values:
            summarize.at[j,i] = sum(results[i]==j)
    del summarize['user']
    return summarize

In [21]:
s = get_results_from_keywords(comp_col='arch', based_on='score', ix=4)
s

{}
Remaining experiments: 9216


C:\Users\marsa\Miniconda3\envs\tesis-env\lib\site-packages\pandas\core\computation\expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
C:\Users\marsa\Miniconda3\envs\tesis-env\lib\site-packages\pandas\core\computation\expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
C:\Users\marsa\Miniconda3\envs\tesis-env\lib\site-packages\pandas\core\computation\expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)
C:\Users\marsa\Miniconda3\envs\tesis-env\lib\site-packages\pandas\core\computation\expressions.py:68: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return op(a, b)


,rank1,rank2,rank3,rank4
rnn,16,8,7,17
cnn,11,10,16,11
tcn,14,14,13,7
mlp,7,16,12,13
